In probabilistic programming languages (PPLs), one needs to compute the joint probability (often unnormalized) of values and observed variables under a generative model to perform approximate inference. However, given a model in the form of a Python function, how does one translate this function (model) into a log joint probability? The objective of this blog is to better understand how modern PPLs, in particular NumPyro, performs this translation in a dynamic way, i.e., the functions for performing this translation can handle a variety of models defined by the user.

In [ ]:
#| code-fold: true
import numpyro